In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import os

# ========== Parameter ==========
FREQ = 52                   # For annualization, use 52 periods per year
TARGET_MEAN_ANNUAL = 0.20   # targeted annualized mean
DATA_PATH = "spx_returns_weekly.xlsx" 
RISK_FREE = 0.0             # risk free
LOWER, UPPER = -0.20, 0.35  # constrain the weights

In [2]:
# ========== Load Data ==========

selected_tickers = ['AAPL', 'NVDA', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'AVGO', 'BRK/B', 'LLY']

df_spx = pd.read_excel(DATA_PATH, sheet_name="s&p500 rets")
if 'date' in df_spx.columns:
    df_spx = df_spx.set_index('date')

cols_spx = [c for c in df_spx.columns if c in selected_tickers]
df_spx = df_spx[cols_spx]

df_bench = pd.read_excel(DATA_PATH, sheet_name="benchmark rets")
if 'date' in df_bench.columns:
    df_bench = df_bench.set_index('date')
df_bench = df_bench[['SPY']]

df = pd.concat([df_spx, df_bench], axis=1, join="inner")
df = df.fillna(method='ffill').dropna(axis=0, how='any')


tickers = df.columns.tolist()
n = len(tickers)
print(tickers)

['AAPL', 'AMZN', 'AVGO', 'BRK/B', 'GOOGL', 'LLY', 'META', 'MSFT', 'NVDA', 'TSLA', 'SPY']


/var/folders/9v/8tx98m4n5db6vcl51bjzg44m0000gn/T/ipykernel_2042/1658888038.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').dropna(axis=0, how='any')


In [3]:
df

,AAPL,AMZN,AVGO,BRK/B,GOOGL,LLY,META,MSFT,NVDA,TSLA,SPY
date,,,,,,,,,,,
2015-01-09,0.024506,-0.037534,0.048060,0.002011,-0.054449,-0.001856,-0.009043,0.009195,-0.009313,-0.057685,-0.005744
2015-01-16,-0.053720,-0.020880,-0.010299,-0.001739,0.019475,0.010725,-0.032933,-0.020130,0.001044,-0.065760,-0.012827
2015-01-23,0.065942,0.074431,0.030476,-0.000603,0.061662,0.020516,0.035241,0.020328,0.037563,0.042575,0.016565
2015-01-30,0.036999,0.134900,-0.038351,-0.034938,-0.008091,-0.001803,-0.024668,-0.143704,-0.072808,0.011476,-0.026931
2015-02-06,0.019120,0.055737,0.018127,0.043569,-0.006854,-0.022779,-0.018969,0.049751,0.062256,0.067589,0.030584
...,...,...,...,...,...,...,...,...,...,...,...
2024-06-14,0.079232,-0.003473,0.233463,-0.019772,0.014519,0.033483,0.023735,0.044167,0.091016,0.002986,0.016423
2024-06-21,-0.023531,0.029511,-0.044040,0.010061,0.016064,0.006181,-0.018605,0.016291,-0.040264,0.028088,0.006424
2024-06-28,0.015085,0.022054,-0.028941,-0.006884,0.014029,0.024325,0.019079,-0.006292,-0.023939,0.081252,-0.000533


In [4]:
# ========== Calculate Annual Mean and Annual Cov ==========
mean_weekly = df.mean()
cov_weekly = df.cov()

mean_annual = mean_weekly * FREQ
cov_annual = cov_weekly * FREQ

target_mean_weekly = TARGET_MEAN_ANNUAL / FREQ

print("---------------------------Annual Mean---------------------------")
print(mean_annual)
print("---------------------------Annual Cov---------------------------")
print(cov_annual)

---------------------------Annual Mean---------------------------
AAPL     0.273082
AMZN     0.311749
AVGO     0.380772
BRK/B    0.127581
GOOGL    0.241664
LLY      0.327800
META     0.255603
MSFT     0.282088
NVDA     0.687172
TSLA     0.461086
SPY      0.136947
dtype: float64
---------------------------Annual Cov---------------------------
           AAPL      AMZN      AVGO     BRK/B     GOOGL       LLY      META  \
AAPL   0.075194  0.040112  0.050696  0.021979  0.041795  0.015291  0.040630   
AMZN   0.040112  0.092360  0.036636  0.016883  0.049418  0.012342  0.053097   
AVGO   0.050696  0.036636  0.116037  0.023928  0.039237  0.013837  0.042694   
BRK/B  0.021979  0.016883  0.023928  0.036061  0.020900  0.015933  0.020087   
GOOGL  0.041795  0.049418  0.039237  0.020900  0.076851  0.014862  0.051170   
LLY    0.015291  0.012342  0.013837  0.015933  0.014862  0.070498  0.012572   
META   0.040630  0.053097  0.042694  0.020087  0.051170  0.012572  0.123100   
MSFT   0.040212  0.04419

In [5]:
# ========== Single-asset Annual Statistics ==========
asset_annual_std = np.sqrt(np.diag(cov_annual)) 
asset_sharpes = (mean_annual - RISK_FREE) / asset_annual_std
asset_summary = pd.DataFrame({
    'mean_annual': mean_annual,
    'std_annual': asset_annual_std,
    'sharpe': asset_sharpes
}).sort_values('sharpe', ascending=False)

asset_summary

,mean_annual,std_annual,sharpe
NVDA,0.687172,0.450385,1.525742
LLY,0.327800,0.265515,1.234585
MSFT,0.282088,0.238037,1.185060
AVGO,0.380772,0.340643,1.117806
AMZN,0.311749,0.303908,1.025803
AAPL,0.273082,0.274215,0.995870
GOOGL,0.241664,0.277220,0.871742
SPY,0.136947,0.168682,0.811866
TSLA,0.461086,0.579716,0.795365
META,0.255603,0.350856,0.728513


# Optimization
$$
\begin{aligned}
\text{Minimize: } &\quad f(w) = w^T \Sigma w \\
\text{subject to: } &\quad \sum_i w_i = 1 \\
&\quad w^T \mu = \mu^*_{\text{target}} \\
&\quad -0.20 \le w_i \le 0.35
\end{aligned}
$$

In [6]:
# ========== Optimization ==========
x0 = np.ones(n) / n       # initial guess weight

# minimize portfolio variance
def obj_var_weekly(w, cov_mat):
    return float(w @ cov_mat @ w)

# The expected return of the portfolio must equal a given target rate of return
cons = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0},
    {'type': 'eq', 'fun': lambda w, m=mean_weekly.values, t=target_mean_weekly: float(np.dot(w, m) - t)}
] 

bounds = tuple((LOWER, UPPER) for _ in range(n))

In [7]:
def port_stats(weights, mean_vec, cov_mat):
    w = np.array(weights)
    port_mean = w.dot(mean_vec)
    port_var = obj_var_weekly(w, cov_mat)
    port_std = np.sqrt(port_var)
    port_sharpe = (port_mean - RISK_FREE) / port_std if port_std > 0 else np.nan
    return port_mean, port_std, port_sharpe

In [8]:
# ========== Bounded 0ptimization ==========
res_bounded = minimize(
    fun=lambda w: obj_var_weekly(w, cov_weekly.values),
    x0=x0,
    method='SLSQP',
    bounds=bounds,
    constraints=cons,
    options={'ftol':1e-12, 'maxiter':1000}
)

if not res_bounded.success:
    print('Bounded optimization warning:', res_bounded.message)

w_bounded = res_bounded.x
mean_bounded_weekly, std_bounded_weekly, sharpe_bounded = port_stats(w_bounded, mean_weekly.values, cov_weekly.values)
mean_bounded_annual = mean_bounded_weekly * FREQ
std_bounded_annual = std_bounded_weekly * np.sqrt(FREQ)

In [9]:
# ========== Unbounded 0ptimization ==========
res_unbounded = minimize(
    fun=lambda w: obj_var_weekly(w, cov_weekly.values),
    x0=x0,
    method='SLSQP',
    bounds=None,
    constraints=cons,
    options={'ftol':1e-12, 'maxiter':1000}
)

if not res_unbounded.success:
    print('Unbounded optimization warning:', res_unbounded.message)

w_unbounded = res_unbounded.x
mean_unbounded_weekly, std_unbounded_weekly, sharpe_unbounded = port_stats(w_unbounded, mean_weekly.values, cov_weekly.values)
mean_unbounded_annual = mean_unbounded_weekly * FREQ
std_unbounded_annual = std_unbounded_weekly * np.sqrt(FREQ)

### 1.1

- Report the weights of the constrained portfolio.
- Report the mean, volatility, and Sharpe ratio of the resulting portfolio.

In [10]:
# ========== 1.1 ==========
bounded_df = pd.DataFrame({
    'ticker': tickers,
    'weight_bounded': w_bounded,
    'asset_mean_annual': mean_annual.values,
    'asset_std_annual': asset_annual_std,
    'asset_sharpe': asset_sharpes.values
}).set_index('ticker')

print('Bounded weights:')
print(bounded_df['weight_bounded'].round(6))
print(f"Bounded portfolio annual mean: {mean_bounded_annual:.6f}")
print(f"Bounded portfolio annual vol: {std_bounded_annual:.6f}")
print(f"Bounded portfolio annual Sharpe (rf=0): {(mean_bounded_annual-RISK_FREE)/std_bounded_annual:.6f}")

Bounded weights:
ticker
AAPL     0.037381
AMZN     0.103109
AVGO     0.046104
BRK/B    0.255105
GOOGL   -0.004611
LLY      0.237236
META     0.000537
MSFT     0.081129
NVDA    -0.025994
TSLA    -0.034685
SPY      0.304688
Name: weight_bounded, dtype: float64
Bounded portfolio annual mean: 0.200000
Bounded portfolio annual vol: 0.159761
Bounded portfolio annual Sharpe (rf=0): 1.251872


### 1.2

- Compare these weights to the assets’ Sharpe ratios and means.
- Do the most extreme positions also have the most extreme Sharpe ratios and means?
- Why?

In [11]:
# ========== 1.2 ==========
extreme_long = bounded_df['weight_bounded'].idxmax()
extreme_short = bounded_df['weight_bounded'].idxmin()

print(f"Most extreme long position: {extreme_long}, weight = {bounded_df.loc[extreme_long,'weight_bounded']:.4f}")
print(f"Most extreme short position: {extreme_short}, weight = {bounded_df.loc[extreme_short,'weight_bounded']:.4f}")

bounded_df

Most extreme long position: SPY, weight = 0.3047
Most extreme short position: TSLA, weight = -0.0347


,weight_bounded,asset_mean_annual,asset_std_annual,asset_sharpe
ticker,,,,
AAPL,0.037381,0.273082,0.274215,0.995870
AMZN,0.103109,0.311749,0.303908,1.025803
AVGO,0.046104,0.380772,0.340643,1.117806
BRK/B,0.255105,0.127581,0.189899,0.671838
GOOGL,-0.004611,0.241664,0.277220,0.871742
LLY,0.237236,0.327800,0.265515,1.234585
META,0.000537,0.255603,0.350856,0.728513
MSFT,0.081129,0.282088,0.238037,1.185060
NVDA,-0.025994,0.687172,0.450385,1.525742


#### Why
- Extreme weights are influenced both by individual asset expected returns (means) and by the covariance structure with other assets.
- An asset with a high Sharpe may receive a large long position, but if it is highly correlated with others or cannot help reduce portfolio variance while meeting the mean constraint, it might not become extreme.
- Conversely, an asset with a modest Sharpe but low correlation to others can be useful for variance reduction and therefore receive larger weight.
- Bounds (-20% to 35%) also truncate what would be extreme weights in unconstrained optimization, so the observed extremes may be at the bounds rather than 'natural' unconstrained values.

### 1.3

- Compare the bounded portfolio weights to the unbounded portfolio weights (obtained from optimizing without the inequality constraints, keeping the equality constraints.)
- Report the mean, volatility, and Sharpe ratio of both.

In [12]:
# ========== 1.3 ==========
unbounded_df = pd.DataFrame({
    'ticker': tickers,
    'weight_unbounded': w_unbounded,
    'asset_mean_annual': mean_annual.values,
    'asset_std_annual': asset_annual_std,
    'asset_sharpe': asset_sharpes.values
}).set_index('ticker')

comparison = bounded_df[['weight_bounded']].join(unbounded_df[['weight_unbounded']])

print('Compare the bounded portfolio weights to the unbounded portfolio weights')
print(comparison)


print('Bounded portfolio stats:')
print(f"Annual mean = {mean_bounded_annual:.6f}, annual vol = {std_bounded_annual:.6f}, Sharpe = {(mean_bounded_annual-RISK_FREE)/std_bounded_annual:.6f}")
print('Unbounded portfolio stats:')
print(f"Annual mean = {mean_unbounded_annual:.6f}, annual vol = {std_unbounded_annual:.6f}, Sharpe = {(mean_unbounded_annual-RISK_FREE)/std_unbounded_annual:.6f}")

Compare the bounded portfolio weights to the unbounded portfolio weights
        weight_bounded  weight_unbounded
ticker                                  
AAPL          0.037381          0.037381
AMZN          0.103109          0.103109
AVGO          0.046104          0.046104
BRK/B         0.255105          0.255105
GOOGL        -0.004611         -0.004611
LLY           0.237236          0.237236
META          0.000537          0.000537
MSFT          0.081129          0.081129
NVDA         -0.025994         -0.025994
TSLA         -0.034685         -0.034685
SPY           0.304688          0.304688
Bounded portfolio stats:
Annual mean = 0.200000, annual vol = 0.159761, Sharpe = 1.251872
Unbounded portfolio stats:
Annual mean = 0.200000, annual vol = 0.159761, Sharpe = 1.251872
